In [1]:
%%time
# Importing all the necessary libraries for this project.

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

CPU times: user 1.14 s, sys: 273 ms, total: 1.41 s
Wall time: 1.69 s


In [29]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [3]:
rawdata = pd.read_json("https://s3.amazonaws.com/temp-data-pulls/newdump.json")

In [8]:
rawdata.head(5)

,brand,channel_info,content,engagement,has_spend,id,impact,share_token,timestamp,type,urls
0,137316,"{'type': ['link'], 'channel': 'facebook'}","{'reactions': {'wow_count': 0, 'sad_count': 0, 'angry_count': 0, 'love_count': 2, 'haha_count': 0}, 'permalink': 'https://www.facebook.com/26815555478/posts/10155699257305479', 'hashtags': '', 'media_name': '11 Women Share What Really Happens to Your Sex Drive During Pregnancy', 'media_caption': 'glamour.com', 'media_description': ' ', 'link_url': 'http://glmr.co/jg9jPXX', 'picture_url': 'https://external.xx.fbcdn.net/safe_image.php?d=AQDhXtx7-b1diuab&url=https%3A%2F%2Fmedia.glamour.com%2Fphotos%2F57a0bc4354c640ad26366526%2Fmaster%2Fpass%2Fknocked-up-in-bed.jpg&_nc_hash=AQB8Ixdn1SF2yWol', 'links': ['http://glmr.co/jg9jPXX', 'glamour.com'], 'post_id': '26815555478_10155699257305479', 'comment_count': 1, 'like_count': 6, 'content_type': 2, 'message': 'Sometimes it gets really, really good.', 'share_count': 0}",9,0.0,MTM3MzE2LTU0NTA4NDg5X2ZhY2Vib29rIHBvc3RfMTc0ODM,0.005619,FRI3zHcgR410PLLtjinjwU-xhf8kQw0VfV3KIISpuwToreqw3fR0cEsVe5xsj9xR,2017-11-16 04:59:09,facebook post,"[{'url': 'http://glmr.co/jg9jPXX', 'path': '/jg9jPXX', 'hostname': 'glmr.co', 'utm': {'term': None, 'medium': None, 'campaign': None, 'content': None, 'source': None, 'id': None}}, {'url': 'https://www.glamour.com/story/11-women-reveal-how-pregnancy-affected-their-sex-lives?mbid=social_facebook_fanpage', 'path': '/story/11-women-reveal-how-pregnancy-affected-their-sex-lives', 'hostname': 'www.glamour.com', 'utm': {'term': None, 'medium': None, 'campaign': None, 'content': None, 'source': None, 'id': None}}]"
1,137299,"{'type': ['article'], 'channel': 'pinterest'}","{'pinned_from': None, 'description': 'Groomed arched eyebrows create a flattering frame for the eyes. And anything that accentuates the eyes is going to make them appear larger.', 'links': [], 'repin_count': 0, 'pin_id': '246290673355306588', 'hashtags': '', 'comment_count': 0, 'like_count': 0, 'post_type': 'article', 'image_url': 'https://s-media-cache-ak0.pinimg.com/originals/9f/9c/9e/9f9c9e89832f73a52848a3ee10370f66.jpg', 'pin_url': 'http://pinterest.com/pin/246290673355306588/'}",0,NaN,MTM3Mjk5LTEzNzEzMDIxX3Bpbl8xNzQ4Mw,NaN,nJ1zxu1n4vEROp9z02MmMkgCRRNvsW1sF16YTlQj-6Y,2017-11-16 04:56:16,pin,NaN
2,137321,"{'type': ['photo', 'link'], 'channel': 'twitter'}","{'link_title': 'Subscribe to SELF’s Newsletter', 'hashtags': '', 'links': ['https://t.co/lMJXiK49Kf', 'https://t.co/apDUJKJ90g'], 'text': 'Did you know? We have a newsletter dedicated to healthy eating! Subscribe here: https://t.co/lMJXiK49Kf https://t.co/apDUJKJ90g', 'link_url': 'https://t.co/lMJXiK49Kf', 'link_description': '', 'tweet_id': '931023079161122816', 'tweet_url': 'https://www.twitter.com/SELFmagazine/status/931023079161122816', 'link_thumbnail': None, 'retweet_count': 3, 'favorite_count': 12, 'media_url': 'https://pbs.twimg.com/tweet_video_thumb/DOuohInUMAAR_7g.jpg'}",15,NaN,MTM3MzIxLTgzMjQ1OTcwX3R3ZWV0XzE3NDgz,1.101843,jNfpfZqFPpHoDgG98xDnzRKGCIbDl8Cuy56G3Acs6Yc,2017-11-16 04:56:14,tweet,NaN
3,137326,"{'type': ['link'], 'channel': 'twitter'}","{'link_title': 'A Peek Inside Tiffany & Co.’s New Café, Where You Can Yes, Eat Breakfast at Tiffany’s', 'hashtags': '', 'links': ['https://t.co/X4JJ6wf7Nj'], 'text': 'Yes, @TiffanyAndCo now has a restaurant on its fourth floor, the Blue Box Café: https://t.co/X4JJ6wf7Nj', 'link_url': 'https://t.co/X4JJ6wf7Nj', 'link_description': 'Get your croissant and coffee orders ready.', 'tweet_id': '931022961557016576', 'tweet_url': 'https://www.twitter.com/voguemagazine/status/931022961557016576', 'link_thumbnail': 'https://assets.vogue.com/photos/5a060aa9049f3e1509b41919/16:9/pass/00-social-image-new-blue-box-cafe-at-tiffany-and-co-fifth-avenue-flagship-store.jpg', 'retweet_count': 49, 'favorite_count': 169, 'media_url': None}",218,NaN,MTM3MzI2LTgzMjQ3MDA0X3R3ZWV0XzE3NDgz,1.574531,G04eObKwGCH2I4rE_U912xKGCIbDl8Cuy56G3Acs6Yc,2017-11-16 04:55:46,tweet,NaN
4,13732

In [32]:
def clean_datatable(rawdata):
    
    ## USING ONLY PIN & BLOG POSTS - RAW DATA HERE
  
    rawpin_blog = rawdata[(rawdata["type"]=="pin") | (rawdata["type"]=="blog post")]
    rawpin_blog.drop(["has_spend"], axis = 1, inplace=True) 
    
    # SEPARATING OUT THE COLUMNS

    channel_info = rawpin_blog['channel_info'].apply(pd.Series)
    channel_info.columns = ["channel", "info"]
    
    # SEPARATING OUT THE COLUMNS THE CONTENT_INFO COLUMN

    content_info = rawpin_blog['content'].apply(pd.Series)
    content_info.drop(['author_email', 'content', 'pinned_from'], axis=1, inplace=True) ## THESE HAVE ONLY NULLS
    
    
    # FIX NAN VALUES IN EACH COLUMN THAT CONTAINS COUNT

    for x in content_info.columns:
        if "count" in x:
            content_info[x].fillna(np.NaN, inplace = True)
        
    # JOINING THE DATASETS ONTO THE RAW

    master_pinblog = rawpin_blog.join(channel_info).join(content_info)
    master_pinblog.drop(['channel_info', 'content'], axis = 1, inplace = True)
    
    # FIX COLUMN NAMES & CLEAN UP TITLES
    
    master_pinblog.columns = ['brand', 'engagement', 'uniqueid', 'impact', 'share_token', 'timestamp',
       'type', 'urls', 'channel', 'info', 'author_name', 'comment_count',
       'description', 'fb_likecount', 'fb_sharecount',
       'gplus_count', 'hashtags', 'image_url', 'like_count',
       'link', 'linkedin_sharecount', 'links', 'pin_id', 'pin_url',
       'pin_count', 'post_type', 'repin_count', 'summary',
       'thumbnail_url', 'title', 'tweet_count']

    return master_pinblog

In [33]:
def blogsplit(master_pinblog):
    
        
    # create new df called blogs that only contains blogs
    blogs = master_pinblog[master_pinblog.type == 'blog post']
    # converts link to string so we can split
    blogs.link = blogs.link.astype(str)
    # instantiate a new list called new_mag
    new_mag = []
    # list comprehension that just keeps part before ‘.com’
    # we can use list comprehension because this is true for all values
    magazine = [i.split('.com')[0] for i in blogs.link]
    # start for loop to get rid of everything before the name of the magazine
    for i in magazine:
        if '.' in i:
            new_mag.append(i.split('.')[1])
    # if there isn’t a ‘.’ it just sends the existing name to the list
        else:
            new_mag.append(i)
    # create new column for the df with the publications
    blogs['pub'] = new_mag
    
    return blogs

In [42]:
def scraper_links(url):
    article = ""
    domain = url.split()[1]
    url = url.split()[0]
    print(url)
    
    return
    ## TRY TO SEE IF THE LINK EXISTS. ELSE JUST RETURN NONE AND MOVE ON.
    try:
        request = requests.get(url)
        soup = BeautifulSoup(request.text, "lxml")
        info = json.loads(soup.find('script', type='application/ld+json').text)
    except:
        article = "Link is Dead"
        other_info = {'headline': None, 'image': None, 'description': None, 'keywords': None, '@type': None}
        return(article, other_info)

    ## IF LINK WORKS, USE JSON ELSE USE BEAUTIFUL SOUP TO GET DATA.
    try:
        article = info["articleBody"]
    except:
        try:
            if domain == "glamour":
                for posts in soup.find_all('div', {'class' : 'article__content'}):
                    article = posts.text
            elif domain == "teenvogue":
                for posts in soup.find_all('div', {'class' : 'content article-content'}):
                    article = posts.text
            elif domain == "wmagazine":
                for posts in soup.find_all('div', {'class' : 'body-text'}):
                    article = posts.text
            elif domain == "allure":
                for posts in soup.find_all('div', {'class' : 'article-body'}):
                    article = posts.text
            elif domain == "cntraveler":
                for posts in soup.find_all('div', {'class' : 'article-body'}):
                    article = posts.text
            elif domain == "architecturaldigest":
                for posts in soup.find_all('div', {'class' : 'article-content-main'}):
                    article = posts.text
            elif domain == "vogue":
                for posts in soup.find_all('div', {'class' : 'article-copy--body'}):
                    article = posts.text
        except:
            article = "Article Not Found"

            
    ## USE JSON TO EXTRACT IMAGE INFO & OTHER STUFF ELSE RETURN NONE
    
    try:        
        attributes = { 'headline', 'image', 'description', 'keywords', '@type'}
        other_info = { k:v for k,v in info.items() if k in attributes }
    except:
        other_info = { 'headline': None, 'image': None, 'description': None, 'keywords': None, '@type': None}
   

    return(article, other_info)

In [43]:
blogs.reset_index(drop=True, inplace=True)
blogs["links + pub"] = blogs['link'] + str(" ") + blogs['pub']

In [14]:
%%time

blogs["story"] = blogs["links + pub"].apply(scraper_links)


KeyError: ('link', 'occurred at index link')

In [48]:
blogs = blogs.join(blogs['story'].apply(pd.Series))
blogs.drop(['story', 'links + pub'], axis=1, inplace = True)

In [57]:
blogs.columns.values[-1] = 'blogdetails'
blogs.columns.values[-2] = 'blogpost'

In [60]:
blogs.to_csv('BlogMaster.csv', mode='a', header=True)